# Imports

In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Q1. Downloading the data

In [3]:
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')

print(f"Number of columns for Jan: {len(df.columns)}")



Number of columns for Jan: 19


# Q2. Computing duration

In [4]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)

std = df.duration.std()

print(f"Standard Deviation of duration : {std}")

Standard Deviation of duration : 46.44530513776499


# Q3. Dropping outliers


In [5]:
before = len(df.duration)

df = df[(df.duration >= 1) & (df.duration <= 60)]

after = len(df.duration)

print(f"Fraction of records left after dropping the outliers : {after / before * 100}")

Fraction of records left after dropping the outliers : 98.27547930522405


# Q4. One-hot encoding


In [6]:
categorical = ["PULocationID", "DOLocationID"]

df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(f"Dimensionality of the matrix : {X_train.shape[1]}")

Dimensionality of the matrix : 515


# Q5. Training a model


In [7]:
target = 'duration'
y_train = df[target].values


In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(f"RMSE Train : {mean_squared_error(y_train, y_pred, squared=False)}")


RMSE Train : 6.9861908469917315


# Q6. Evaluating the model


In [9]:
df = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ["PULocationID", "DOLocationID"]

df[categorical] = df[categorical].astype(str)

val_dicts = df[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df[target].values

y_pred = lr.predict(X_val)

print(f"RMSE Validation : {mean_squared_error(y_val, y_pred, squared=False)}")


RMSE Validation : 7.786406838555644
